In [68]:
import os
os.chdir('/Documents/CBA_Stats')

from cbastats import DBHelper
import datetime
import pandas as pd
from tqdm.notebook import tqdm
from collections import defaultdict
from bson import objectid

# Connect to Database

In [151]:
mongodbio =DBHelper.MongoDBHelper() 
client = mongodbio.create_connection(DBHelper.MONGODB_USERNAME,DBHelper.MONGODB_PWD,DBHelper.MONGODB_ENDPOINT)

db = client['cbaStats']

existing database ['cbaStats', 'admin', 'local']


In [152]:
db.list_collection_names()

['cbaAllGamesBoxScore',
 'cbaGamesStaging',
 'uniqueBoxScores',
 'seasonTeamStats',
 'seasonPlayerStats',
 'cbaGames']

In [153]:
coll_cbaGames = db['cbaGames']

# pull games stats

In [155]:
coll_cbaAllGamesBoxScore = db['cbaAllGamesBoxScore']

all_box_scores=pd.DataFrame(mongodbio.select_records(coll_cbaAllGamesBoxScore,{}))
all_box_scores.shape

# Get Unique BoxScore

In [160]:
simple_boxscores = all_box_scores[['篮板', '助攻','抢断','盖帽','得分']].copy()

uniq_simple_boxscores = simple_boxscores.drop_duplicates().sort_values(by='得分',ascending=False).reset_index(drop=True)

uniq_simple_boxscores = uniq_simple_boxscores.reset_index()

#删除所有数据统计为0 的数据
uniq_simple_boxscores.drop(uniq_simple_boxscores.loc[uniq_simple_boxscores[['篮板', '助攻', '抢断', '盖帽', '得分']].sum(axis=1)==0].index,inplace=True)

unique_boxscores = uniq_simple_boxscores.merge(all_box_scores,on=['篮板', '助攻', '抢断', '盖帽', '得分'])
unique_boxscores

In [165]:
unique_boxscores.columns

Index(['index', '篮板', '助攻', '抢断', '盖帽', '得分', '_id', '球员_link', '球员', '号码',
       '出场时间', '首发', '进攻篮板', '防守篮板', '犯规', '失误', '扣篮', '被侵', '快攻', '轮次', '日期',
       '赛季', 'GameID_Sina', '球队ID', '对手ID', '球队', '对手', '地点', '2分中', '2分投',
       '3分中', '3分投', '罚球中', '罚球投'],
      dtype='object')

In [166]:
occurrences = uniq_simple_boxscores.to_dict('record')

for row in tqdm(occurrences):
    row['事件'] = unique_boxscores.loc[unique_boxscores['index']==row['index']].to_dict('record')

In [167]:
def batch_list(lst:list, batch_size=1):
    l = len(lst)
    for ndx in range(0, l, batch_size):
        yield lst[ndx:min(ndx + batch_size, l)]

In [116]:
for x in batch_list([1,2,3,4,5,6,7,8,9,0],4):
    print(x)

[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 0]


In [168]:
coll_uniqueBoxScores = db['uniqueBoxScores']

mongodbio.delete_records(coll_uniqueBoxScores,{})
bad_batch = []
for batch in tqdm(batch_list(occurrences,1000)):
    try:
        mongodbio.insert_records(coll_uniqueBoxScores,batch)
    except:
        bad_batch.extend(batch)

uniqueBoxScores operation acknowledged!
29078 records were deleted from uniqueBoxScores .


uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores opera

# Get Data

In [148]:
mongodbio.select_records(coll_uniqueBoxScores,{},nlimit=10)

[{'_id': ObjectId('60066a6a4d892ff35d11ae0d'),
  'index': 0,
  '篮板': 10.0,
  '助攻': 4.0,
  '抢断': 2.0,
  '盖帽': 0.0,
  '得分': 82.0,
  '事件': [{'index': 0,
    '篮板': 10.0,
    '助攻': 4.0,
    '抢断': 2.0,
    '盖帽': 0.0,
    '得分': 82.0,
    '球员_link': 'http://cba.sports.sina.com.cn/cba/player/show/4834/',
    '球员': '麦克鲁姆',
    '号码': '3',
    '出场时间': 48.0,
    '首发': 1.0,
    '进攻篮板': 2.0,
    '防守篮板': 8.0,
    '犯规': 5.0,
    '失误': 5.0,
    '扣篮': 1.0,
    '被侵': 20.0,
    '快攻': '',
    '轮次': '第37轮',
    '日期': datetime.datetime(2015, 1, 30, 11, 35),
    '赛季': '14-15',
    'GameID_Sina': '13056',
    '球队ID': '6',
    '对手ID': '1',
    '球队': '浙江',
    '对手': '广东',
    '地点': '',
    '2分中': 21.0,
    '2分投': 35.0,
    '3分中': 5.0,
    '3分投': 14.0,
    '罚球中': 25.0,
    '罚球投': 27.0}]},
 {'_id': ObjectId('60066a6a4d892ff35d11ae0e'),
  'index': 1,
  '篮板': 1.0,
  '助攻': 5.0,
  '抢断': 8.0,
  '盖帽': 0.0,
  '得分': 75.0,
  '事件': [{'index': 1,
    '篮板': 1.0,
    '助攻': 5.0,
    '抢断': 8.0,
    '盖帽': 0.0,
    '得分': 75.0,
    